# 3. PPO KL 페널티 비교 실험

이 노트북은 사전 학습된 SFT, RM 모델을 불러와 PPO 단계만 실행합니다.
KL 페널티 계수(`kl_coef`) 리스트 `[0.0, 0.1, 0.5]`에 대해 순차적으로 학습과 평가를 모두 수행하고, 마지막에 결과를 하나의 표로 요약합니다.

## 1. 라이브러리 임포트 및 로그인

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer
import json
import wandb
from typing import Dict, List
from copy import deepcopy
import gc

from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.models.base import RewardModel
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy

wandb.login()

## 2. 실험 파라미터 및 데이터 준비

In [ ]:
# =========== 실험할 KL 페널티 리스트 ===========
KL_VALUES = [0.0, 0.1, 0.5]
# ======================================================

# 불러올 SFT/RM 모델의 버전 (v2.5 사용)
BASE_VERSION_NAME = 'v2.5' 

# PPO 파라미터
PPO_NUM_EPISODES = 100
PPO_MAX_EPOCHS = 3

# 모델 및 데이터 경로
BASE_MODEL_NAME = 'skt/kogpt2-base-v2'
SFT_MODEL_NAME = f'models/sft_output_model_{BASE_VERSION_NAME}'
RM_MODEL_NAME = f'models/rm_output_model_{BASE_VERSION_NAME}'

DATA_PATH_3_PPO = f'data/kochatgpt_3_PPO_{BASE_VERSION_NAME}.jsonl'

# 토크나이저 및 PPO 프롬프트 로드
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL_NAME, bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512
)

with open(DATA_PATH_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

print(f"실험할 KL 계수: {KL_VALUES}")

In [ ]:
from chatgpt.trainer.callbacks import Callback
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# wandb 로깅과 loss 기록을 위한 Custom Callback 정의
class WandbPlottingCallback(Callback):
    def __init__(self):
        super().__init__()
        self.actor_losses = []
        self.critic_losses = []

    def on_learn_batch_end(self, metrics: dict, experience: "Experience") -> None:
        # 학습 스텝이 끝날 때마다 actor와 critic의 loss를 wandb에 기록하고 리스트에 저장
        wandb.log(metrics)
        if 'actor_loss' in metrics:
            self.actor_losses.append(metrics['actor_loss'])
        if 'critic_loss' in metrics:
            self.critic_losses.append(metrics['critic_loss'])


## 3. PPO 실험 루프 실행

In [ ]:
# 결과를 저장할 딕셔너리 초기화
all_results = {}
all_actor_logs = {}
all_critic_logs = {}

# --- 데이터셋에서 평가를 위한 프롬프트 100개 로드 ---
with open(DATA_PATH_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

if len(list_data_dict) > 100:
    test_prompts = [item['prompt'] for item in list_data_dict[-100:]]
else:
    test_prompts = [item['prompt'] for item in list_data_dict]

print(f"총 {len(test_prompts)}개의 프롬프트로 평가를 진행합니다.")

PROMPT_DICT = {"prompt_input": ("### Instruction(명령어):\n{prompt}\n\n### Response(응답):")}
formatted_test_prompts = [PROMPT_DICT['prompt_input'].format_map({'prompt': p}) for p in test_prompts]

def generation(input_text, model, tokenizer):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(torch.cuda.current_device())
    outputs = model.model.generate(input_ids, max_length=250, do_sample=True, top_k=50, top_p=0.95, num_return_sequences=1)
    output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    prompt_part = input_text.split("### Response(응답):")[0] + "### Response(응답):"
    return output_text.replace(prompt_part, "").strip()

# --- 실험 루프 시작 ---
for kl_coef in KL_VALUES:
    print(f"\n======================================================")
    print(f"=== KL_COEF = {kl_coef} 실험 시작 ===")
    print(f"======================================================\n")

    with NaiveStrategy().model_init_context():
        actor = GPTActor(pretrained=SFT_MODEL_NAME, lora_rank=16).to(torch.cuda.current_device())
        critic = GPTCritic(pretrained=RM_MODEL_NAME, lora_rank=8).to(torch.cuda.current_device())

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

    actor_optim = torch.optim.Adam(actor.parameters(), lr=5e-6)
    critic_optim = torch.optim.Adam(critic.parameters(), lr=5e-6)

    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
        (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

    ppo_run_name = f'ppo_kl_{kl_coef}_{BASE_VERSION_NAME}'
    wandb.init(project="kochatgpt_tuning_kl_exp", name=ppo_run_name, reinit=True)

    # 콜백 인스턴스 생성
    wandb_plot_callback = WandbPlottingCallback()

    trainer = PPOTrainer(
        NaiveStrategy(), actor, critic, reward_model, initial_model, actor_optim, critic_optim,
        max_epochs=PPO_MAX_EPOCHS, train_batch_size=8, tokenizer=tokenize_fn, max_length=128,
        do_sample=True, temperature=1.0, top_k=50, kl_coef=kl_coef,
        pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id,
        callbacks=[wandb_plot_callback]  # 콜백 추가
    )

    trainer.fit(list_prompt, num_episodes=PPO_NUM_EPISODES, max_timesteps=3, update_timesteps=3)

    # 학습 기록 저장
    all_actor_logs[kl_coef] = wandb_plot_callback.actor_losses
    all_critic_logs[kl_coef] = wandb_plot_callback.critic_losses

    print(f"\n--- KL={kl_coef} 모델 생성 결과 ---")
    current_results = []
    for p, fp in zip(test_prompts, formatted_test_prompts):
        output = generation(fp, actor, tokenizer)
        current_results.append(output)
    all_results[kl_coef] = current_results

    wandb.finish()
    del actor, critic, initial_model, reward_model, trainer, actor_optim, critic_optim
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
# PPO Actor/Critic Loss 비교 그래프
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Actor Loss 그래프
for kl_coef in KL_VALUES:
    logs = all_actor_logs.get(kl_coef, [])
    if logs:
        steps = range(len(logs))
        ax1.plot(steps, logs, label=f'Actor Loss (kl={kl_coef})')
ax1.set_title('PPO Actor Loss Comparison')
ax1.set_xlabel('Training Steps')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True)

# Critic Loss 그래프
for kl_coef in KL_VALUES:
    logs = all_critic_logs.get(kl_coef, [])
    if logs:
        steps = range(len(logs))
        ax2.plot(steps, logs, label=f'Critic Loss (kl={kl_coef})')
ax2.set_title('PPO Critic Loss Comparison')
ax2.set_xlabel('Training Steps')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## 4. 최종 비교 분석

In [ ]:
from IPython.display import display, Markdown

# 마크다운 테이블 생성
# 동적으로 헤더 생성
header = '| Prompt | ' + ' | '.join([f'PPO (kl={kl})' for kl in KL_VALUES]) + ' |'
separator = '| :--- | ' + ' | '.join([':---' for _ in KL_VALUES]) + ' |'
markdown_table = f'{header}\n{separator}'

# test_prompts는 실험 루프 셀에서 이미 정의되어 있다고 가정합니다.
for i, prompt in enumerate(test_prompts):
    # 프롬프트를 안전하게 처리하고 행 시작
    safe_prompt = prompt.replace('|', '\|')
    row = f'| **{safe_prompt}** '

    # 각 kl_coef 값에 대한 결과를 행에 추가
    for kl in KL_VALUES:
        # 결과 딕셔너리에서 결과 가져오기
        output = all_results.get(kl, [''] * len(test_prompts))[i]
        # 줄바꿈 문자를 HTML <br> 태그로 변경하여 셀 내 줄바꿈 처리
        output_html = output.replace('\n', '<br>')
        row += f'| {output_html} '

    markdown_table += f'\n{row}|'

display(Markdown(markdown_table))